In [4]:
import pandas as pd
from geopy.distance import geodesic
import numpy as np
import matplotlib.pyplot as plt
import pickle
from datetime import datetime, timedelta

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.cluster import KMeans
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.utils import resample
from sklearn.svm import SVC


clean data. make rolling window

In [221]:
fd = pd.read_pickle('data/on_faults.pkl')

In [222]:
fd['EventTimeStamp'] = pd.to_datetime(fd['EventTimeStamp'])

In [223]:
fd = fd.loc[fd['active'] == True]

In [224]:
fd = fd.replace({'unknown': '0'})

In [225]:
fd_time = fd['EventTimeStamp']

In [226]:
fd

,RecordID,ESS_Id,EventTimeStamp,eventDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,...,LampStatus,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure,dist_A,dist_B,dist_C
0,1,990349,2015-02-21 10:47:13,Low (Severity Low) Engine Coolant Level,0,0,0,0,0,111,...,1023,True,NaN,0,3276.75,NaN,0,216.606433,246.704143,200.193116
1,2,990360,2015-02-21 11:34:34,NaN,0,0,0,0,11,629,...,1279,NaN,NaN,NaN,NaN,NaN,NaN,216.606433,246.704143,200.193116
3,4,990370,2015-02-21 11:35:33,Incorrect Data Steering Wheel Angle,0,0,0,0,11,1807,...,1279,NaN,NaN,NaN,NaN,NaN,NaN,376.273067,408.644871,437.328302
5,6,990431,2015-02-21 11:40:22,Low (Severity Low) Engine Coolant Level,04993120*00025921*082113134117*07700053*I0*BBZ*,79466580,6X1u10D1500000000,CMMNS,0,111,...,1023,NaN,NaN,13.6022,3276.75,NaN,6.67,593.183546,583.476307,771.563146
6,7,990439,2015-02-21 11:40:52,Low (Severity Low) Engine Coolant Level,0,0,0,0,0,111,...,1023,NaN,NaN,41.53478,3276.75,NaN,20.59,57.661421,90.640855,187.904738
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1057451,1248448,123899434,2020-03-06 13:12:43,High Voltage (Fuel Level),NaN,NaN,CECU3B-NAMUX4,PACCR,49,96,...,1279,False,NaN,0.9417657,NaN,100,1.16,476.861209,450.926748,409.399502
1057454,1248452,123901805,2020-03-06 13:42:48,Low (Severity Medium) Engine Coolant Level,04358814*06030918*051718174436*09401683*G1*BDR*,79904453,6X1u13D1500000000,CMMNS,0,111,...,2047,False,NaN,5.932153,NaN,100,0.58,554.653921,569.483398,389.053864
1057457,1248455,123905139,2020-03-06 14:04:23,Condition Exists Engine Protection Torque Derate,04358814*06099720*030816202706*09400153*G1*BDR*,79932020,6X1u13D1500000000,CMMNS,0,1569,...,18431,False,NaN,65.01096,NaN,73.2,7.83,410.888391,404.516415,243.920462
1057458,1248456,123905996,2020-03-06 14:13:38,Abnormal Rate of Change Aftertreatment 1 Intak...,05317106*05100987*050719120655*09401585*G1*BDR*,79880653,6X1u13D1500000000,CMMNS,0,3216,...,17407,NaN,NaN,66.5741,NaN,100,6.96,141.618964,117.628771,156.616286


In [227]:
fd = fd[['spn','EquipmentID', 'EngineOilTemperature','EngineLoad', 'EngineOilPressure','fmi']]

# Convert the columns to numeric data types
fd = fd.apply(pd.to_numeric, errors='coerce')

# Fill NaN values with 0 and convert to integers
fd = fd.fillna(0).astype(int)

In [228]:
fd = pd.merge(fd_time, fd, left_index=True, right_index=True)

In [230]:
fd_dum =pd.get_dummies(data=fd.loc[:, ['EventTimeStamp',
                                    
                                       'spn',
                                       'EquipmentID', 
                                       'EngineOilTemperature',
                                       'EngineLoad', 
                                       'EngineOilPressure',
                                       'fmi']], 
                           columns=['spn']).sort_values(['EventTimeStamp']).reset_index(drop = True)
                                                            

In [239]:
fd_rolling = (fd_dum
              .groupby('EquipmentID')
                        .rolling(window = '1D', on = 'EventTimeStamp')
                        .sum()
                 )
                        

In [324]:
fd_rolling = fd_rolling.sort_values(by = 'EventTimeStamp')

fd_rolling.head()

EventTimeStamp  EngineLoad  EngineOilPressure   
EquipmentID                                                        
1488        0 2015-02-20 17:51:31         0.0                0.0  \
1439        1 2015-02-21 10:47:13        11.0                0.0   
309         2 2015-02-21 11:14:38         0.0               41.0   
310         4 2015-02-21 11:18:20        56.0               56.0   
            3 2015-02-21 11:18:20        28.0               28.0   

               EngineOilTemperature   fmi  spn_0  spn_100  spn_101  spn_102   
EquipmentID                                                                   
1488        0                   0.0  31.0    0.0      0.0      0.0      0.0  \
1439        1                  96.0  17.0    0.0      0.0      0.0      0.0   
309         2                 193.0   2.0    0.0      0.0      0.0      0.0   
310         4                 384.0   6.0    0.0      0.0      0.0      0.0   
            3                 192.0   3.0    0.0      0.0      0.0      0.0   

               spn_1023  ...  spn_938  spn_939  spn_94  spn_940  spn_941   
EquipmentID              ...                                               
1488        0       0.0  ...      0.0      0.0     0.0      0.0      0.0  \
1439        1       0.0  ...      0.0      0.0     0.0      0.0      0.0   
309         2       0.0  ...      0.0      0.0     0.0      0.0      0.0   
310         4       0.0  ...      0.0      0.0     0.0      0.0      0.0   
            3       0.0  ...      0.0      0.0     0.0      0.0      0.0   

               spn_95  spn_96  spn_97  spn_976  spn_98  
EquipmentID                                             
1488        0     0.0     0.0     0.0      0.0     0.0  
1439        1     0.0     0.0     0.0      0.0     0.0  
309         2     0.0     0.0     0.0      0.0     0.0  
310         4     0.0     1.0     0.0      0.0     0.0  
            3     0.0     1.0     0.0      0.0     0.0  

[5 rows x 422 columns]

In [351]:
def filter_window(fd_rolling, event_name, position, window):
    if position == 'first':
        end_date = fd_rolling.loc[(fd_rolling.spn_5246 == 1.0),'EventTimeStamp'].min()
    else:
        end_date = fd_rolling.loc[(fd_rolling.spn_5246 == 1.0),'EventTimeStamp'].max()
    start_date = end_date - pd.Timedelta(window)
    return fd_rolling[(fd_rolling.EventTimeStamp >= start_date) & (fd_rolling.EventTimeStamp <= end_date)]
    
fd_model = filter_window(fd_rolling, 'spn_5246', 'first', '1D')

In [352]:
fd_model


,,EventTimeStamp,EngineLoad,EngineOilPressure,EngineOilTemperature,fmi,spn_0,spn_100,spn_101,spn_102,spn_1023,...,spn_938,spn_939,spn_94,spn_940,spn_941,spn_95,spn_96,spn_97,spn_976,spn_98
EquipmentID,,,,,,,,,,,,,,,,,,,,,,
1401,249,2015-02-22 05:10:11,0.0,0.0,0.0,13.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1606,250,2015-02-22 05:13:14,137.0,172.0,628.0,45.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1595,251,2015-02-22 05:20:45,318.0,199.0,1198.0,119.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1432,252,2015-02-22 05:35:38,0.0,34.0,207.0,18.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1585,253,2015-02-22 05:38:36,52.0,68.0,440.0,51.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1749,991,2015-02-23 04:50:50,134.0,935.0,3956.0,326.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1641,992,2015-02-23 04:53:09,330.0,509.0,2585.0,115.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1419,993,2015-02-23 04:59:17,17.0,34.0,147.0,21.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [327]:
#fd_model  = fd_rolling.loc[~(fd_rolling ['spn_5246'] > 1)]

model 1: RandomForestClassifier

In [330]:
# Separate majority and minority classes
df_majority = fd_model[fd_model.spn_5246==0]
df_minority = fd_model[fd_model.spn_5246==1]
 
# Downsample majority class
df_majority_downsampled = resample(df_majority, 
                                 replace=False,    # sample without replacement
                                 n_samples=2000,     # to match minority class
                                 random_state=200) # reproducible results
 
# Combine minority class with downsampled majority class
df_downsampled = pd.concat([df_majority_downsampled, df_minority])
 
# Display new class counts
df_downsampled.spn_5246.value_counts()

spn_5246
0.0    2000
1.0    1227
Name: count, dtype: int64

In [331]:
X = df_downsampled.drop(columns=['spn_5246', 'spn_1569','EventTimeStamp'])

y = df_downsampled[['spn_5246']]

X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=.25, random_state = 321)

In [332]:
scaler = StandardScaler()
imputer = SimpleImputer()
rfc = RandomForestClassifier  # LogisticRegression

In [333]:
steps = [("imputer", imputer), 
         ("scaler", scaler),
         ("rfc", RandomForestClassifier(n_estimators=100, max_depth = 100
                    )
         )
         
        ]


pipeline = Pipeline(steps = steps)

In [334]:
pipeline.fit(X_train, y_train)

C:\Users\tjsim\Anaconda\lib\site-packages\sklearn\pipeline.py:405: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)


Pipeline(steps=[('imputer', SimpleImputer()), ('scaler', StandardScaler()),
                ('rfc', RandomForestClassifier(max_depth=100))])

In [335]:
print(classification_report(y_test, pipeline.predict(X_test)))

              precision    recall  f1-score   support

         0.0       0.96      0.95      0.96       498
         1.0       0.93      0.94      0.93       309

    accuracy                           0.95       807
   macro avg       0.94      0.95      0.95       807
weighted avg       0.95      0.95      0.95       807



In [336]:
print(confusion_matrix(y_test, pipeline.predict(X_test)))

[[475  23]
 [ 19 290]]


In [353]:
X2 = fd_model.drop(columns=['spn_5246', 'spn_1569','EventTimeStamp']) # make df equipmentID , stratiy on variable of trucks w/ derate

y2 = fd_model['spn_5246'] # equipmentID

X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=.25, random_state = 321)

In [354]:
scaler = StandardScaler()
imputer = SimpleImputer()
rfc = RandomForestClassifier 

In [355]:
steps = [("imputer", imputer), 
         ("scaler", scaler),
         ("rfc", RandomForestClassifier(n_estimators=100, max_depth = 200
                    )
         )
         
        ]


pipeline = Pipeline(steps = steps)

In [356]:
pipeline.fit(X2_train, y2_train)

Pipeline(steps=[('imputer', SimpleImputer()), ('scaler', StandardScaler()),
                ('rfc', RandomForestClassifier(max_depth=200))])

In [357]:
rfc = pipeline.named_steps['rfc']  # get the trained RandomForestClassifier model from the pipeline

importances = pd.DataFrame({
    'variable': X2.columns,       # get importance of variables in model
    'importance': rfc.feature_importances_
})

importances.sort_values('importance', ascending = False).head(7)

,variable,importance
129,spn_3226,0.193737
299,spn_5848,0.143892
291,spn_5743,0.126297
2,EngineOilTemperature,0.107720
300,spn_5851,0.097317
3,fmi,0.070782
321,spn_627,0.065216


In [358]:
print(classification_report(y2_test, pipeline.predict(X2_test)))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00       187

    accuracy                           1.00       187
   macro avg       1.00      1.00      1.00       187
weighted avg       1.00      1.00      1.00       187



In [359]:
print(confusion_matrix(y2_test, pipeline.predict(X2_test)))


[[187]]


In [ ]:
#scores = cross_val_score(rfc, X2, y2, cv=5, scoring='f1')

In [ ]:
# true neg = top left 
# false pos = top right
# false neg = bottom left
# true pos = bottom right 

In [ ]:
#full_derates_seasons = full_derates_seasons[['EventTimeStamp',spn', 'AcceleratorPedal','EngineRpm','EquipmentID', 'EngineOilTemperature',
                                             #'EngineLoad', 'EngineOilPressure','fmi']]

# Convert the columns to numeric data types
#full_derates_seasons = full_derates_seasons.apply(pd.to_numeric, errors='coerce')

# Fill NaN values with 0 and convert to integers
#full_derates_seasons = full_derates_seasons.fillna(0).astype(int)

In [ ]:
#full_derates_seasons=full_derates_seasons.set_index('EventTimeStamp').sort_index(ascending = False)